In [1]:
# State

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_base import ProjectedGradientDescentBase, ProjectedGradientDescentBaseOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import SimulationSetting

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [5]:
# Case 1:
# true_object = get_z0_1q(c_sys)

# Case 2:
vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
true_object = State(c_sys, vec)

# Case 3:
# vec = np.array([1 / np.sqrt(2), 0, 0, 0], dtype=np.float64)
# true_object = State(c_sys, vec)

true_object.vec

array([0.70710678, 0.40824829, 0.40824829, 0.40824829])

In [7]:
num_data = [100, 1000, 10000]
# n_rep = 100
n_rep = 10

case_name_list = [
    "Linear(True)",
    "Linear(False)",
    "ProjectedLinear(True)",
    "ProjectedLinear(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(
        tester_objects,
        on_para_eq_constraint=True,
        on_algo_eq_constraint=True,
        on_algo_ineq_constraint=True,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardQst(
        tester_objects,
        on_para_eq_constraint=False,
        on_algo_eq_constraint=True,
        on_algo_ineq_constraint=True,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardQst(
        tester_objects,
        on_para_eq_constraint=True,
        on_algo_eq_constraint=True,
        on_algo_ineq_constraint=True,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardQst(
        tester_objects,
        on_para_eq_constraint=False,
        on_algo_eq_constraint=True,
        on_algo_ineq_constraint=True,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
]
para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(3),
    WeightedRelativeEntropy(4),
    WeightedProbabilityBasedSquaredError(3),
    WeightedProbabilityBasedSquaredError(4),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption(),
    WeightedRelativeEntropyOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
    WeightedProbabilityBasedSquaredErrorOption(),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
    ProjectedGradientDescentBase(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
    ProjectedGradientDescentBaseOption(),
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        iteration=n_rep,
    )

    # stock settings of this simulation
    simulation_setting = SimulationSetting(
        name=name,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
    )
    simulation_settings.append(simulation_setting)

    estimation_results_list.append(estimation_results)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

 10%|█         | 1/10 [00:00<00:01,  7.14it/s]

Case 0: Linear(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


 10%|█         | 1/10 [00:00<00:01,  8.36it/s]

elapsed_time:0.020361363887786865[min]

Case 1: Linear(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator


  0%|          | 0/10 [00:00<?, ?it/s]

...

elapsed_time:0.28760866324106854[min]

Case 6: Least Squares(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LossMinimizationEstimator


  0%|          | 0/10 [00:00<?, ?it/s]

elapsed_time:0.05923301378885905[min]

Case 7: Least Squares(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LossMinimizationEstimator


100%|██████████| 10/10 [00:03<00:00,  2.55it/s]

elapsed_time:0.06549976666768392[min]



In [192]:
report.export_report("qst_nrep=10_case=1_軸上から一番離れた表面_mle_lse_変更後.pdf",
    estimation_results_list=estimation_results_list,   # 「EstimationResultのリスト」のリスト
    simulation_settings = simulation_settings,
    true_object=true_object,  # True Object
    tester_objects = tester_objects,   # Tester Objectのリスト.
    seed=seed,  # 推定で使ったseed（オプション）
    computation_time=sum(elapsed_times) # 処理時間の合計（オプション）
)

100%|██████████| 10/10 [00:00<00:00, 11519.65it/s]

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars



​​Generating physicality violation test blocks ...


100%|██████████| 10/10 [00:00<00:00, 4198.08it/s]
/Users/tomoko/project/rcast/workspace/quara/quara/data_analysis/physicality_violation_check.py:82: UserWarning:

i=2: invalid: values=[0.9757877678124989, 0.024212232187501292]
less_list=[]
sum=1.0000000000000002, greater_list=[0.9757877678124989, 0.024212232187501292]

/Users/tomoko/project/rcast/workspace/quara/quara/data_analysis/physicality_violation_check.py:82: UserWarning:

i=7: invalid: values=[0.9856469911365666, 0.01435300886343367]
less_list=[]
sum=1.0000000000000002, greater_list=[0.9856469911365666, 0.01435300886343367]

/Users/tomoko/project/rcast/workspace/quara/quara/data_analysis/physicality_violation_check.py:82: UserWarning:

i=9: invalid: values=[0.9879764338572106, 0.012023566142789821]
less_list=[]
sum=1.0000000000000004, greater_list=[0.9879764338572106, 0.012023566142789821]

/Users/tomoko/project/rcast/workspace/quara/quara/data_analysis/physicality_violation_check.py:87: UserWarning:

sum_eig_less_list and sum_

Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (qst_nrep=10_case=1_軸上から一番離れた表面_mle_lse_変更後.pdf)


In [10]:
report.export_report?

Signature:
report.export_report(
    path:str,
    estimation_results_list:List[List[_ForwardRef('EstimationResult')]],
    case_name_list:List[str],
    estimator_list:List[_ForwardRef('Estimator')],
    true_object:'QOperation',
    tester_objects:List[_ForwardRef('QOperation')],
    loss_list:List[_ForwardRef('ProbabilityBasedLossFunction')]=None,
    loss_option_list:List[_ForwardRef('ProbabilityBasedLossFunctionOption')]=None,
    algo_list:List[_ForwardRef('MinimizationAlgorithm')]=None,
    algo_option_list:List[_ForwardRef('MinimizationAlgorithmOption')]=None,
    seed:Union[int, NoneType]=None,
    computation_time:Union[float, NoneType]=None,
    keep_tmp_files:bool=False,
    show_physicality_violation_check:bool=True,
)
Docstring: <no docstring>
File:      c:\users\satoyuki\myspace\github\tknrsgym\quara\quara\data_analysis\report.py
Type:      function
